In [1]:
import pandas as pd
import numpy as np
from gurobipy import *
import openpyxl
from openpyxl.styles import PatternFill
import copy

In [2]:
#data read in 
paper_data =  pd.read_excel('paper_data.xlsx')
paper_data.columns = ['paper'] + list(paper_data.columns[1:])
paper_data[paper_data.columns[:10]]

,paper,Referee1,Referee2,Referee3,Referee4,Referee5,Referee6,Referee7,Referee8,Referee9
0,Paper1,yes,no,no,no,no,yes,no,no,no
1,Paper2,no,no,maybe,maybe,no,yes,no,no,maybe
2,Paper3,no,no,no,yes,yes,no,no,yes,no
3,Paper4,no,no,no,maybe,yes,no,yes,no,no
4,Paper5,no,no,no,no,conflict,no,no,conflict,no
...,...,...,...,...,...,...,...,...,...,...
66,Paper67,maybe,conflict,no,no,no,no,no,no,no
67,Paper68,no,no,yes,no,no,maybe,no,conflict,no
68,Paper69,yes,no,no,maybe,no,no,no,no,no
69,Paper70,no,yes,no,no,maybe,maybe,no,no,no


In [3]:
paper_data_matrix = paper_data.values[0:,1:].copy()
paper_data_matrix[paper_data_matrix=='yes'] = 0
paper_data_matrix[paper_data_matrix=='maybe'] = 1
paper_data_matrix[paper_data_matrix=='no'] = 2
paper_data_matrix[paper_data_matrix=='conflict'] = 1e10

#cost matrix
paper_data_matrix

array([[0, 2, 2, ..., 2, 0, 2],
       [2, 2, 1, ..., 2, 2, 1],
       [2, 2, 2, ..., 2, 1, 1],
       ...,
       [0, 2, 2, ..., 2, 2, 0],
       [2, 0, 2, ..., 2, 1, 0],
       [2, 2, 2, ..., 2, 1, 2]], dtype=object)

In [1]:
# Modelling
myModel = Model( "paper_assignment" )
var_dic_x = {}
var_dic_y = {}
constraint = {}
cost_matrix = paper_data_matrix.T.copy()

#decision varaible 
for i in range(1,22):
    var_dic_x['0' + '_' + str(i)] = myModel.addVar(vtype = GRB.CONTINUOUS , \
                                                   name = "x%s"%('0' + '_' + str(i)), ub = 11)
for i in range(1,22):
    for j in range(1,72):
        var_dic_y[str(i) + '_' +str(j)] = myModel.addVar(vtype = GRB.CONTINUOUS , \
                                               name = "y%s"%(str(i) + '_' + str(j)), ub = 1)
myModel.update()  

#objective function 
objExpr = LinExpr()
for i in range(1,22):
    for j in range(1,72):
        objExpr +=  cost_matrix[i-1][j-1] * var_dic_y[str(i) + '_' +str(j)]
myModel.setObjective(objExpr , GRB.MINIMIZE)
myModel.update()  

#constraint function
Constr_start = LinExpr()
for i in range(1,22):
     Constr_start +=var_dic_x['0' + '_' + str(i)]
myModel.addConstr( lhs = Constr_start , sense = GRB.EQUAL , \
                   rhs = 213 , name = "Const_start-" + str(0) )

for i in range(1,22): 
    constExpr = LinExpr()
    constExpr += -1* var_dic_x['0' + '_' + str(i)]
    for j in range(1,72):
        constExpr += var_dic_y[str(i) + '_' +str(j)]
    myModel.addConstr( lhs = constExpr , sense = GRB.EQUAL , rhs = 0 , \
                       name = "reviewer_node-" + str(i) )
    

for j in range(1,72):
    constExpr = LinExpr()
    for i in range(1,22): 
         constExpr += var_dic_y[str(i) + '_' +str(j)]
    myModel.addConstr( lhs = constExpr , sense = GRB.EQUAL , rhs = 3, \
                       name = "paper_node-" + str(j) )
    

myModel.update()
# write the model in a file to make sure it is constructed correctly
myModel.write( filename = "fl.lp" )

# optimize the model
myModel.optimize()

# check the status of the model
curStatus = myModel.status
if curStatus in (GRB.Status.INF_OR_UNBD, GRB.Status.INFEASIBLE, \
              GRB.Status.UNBOUNDED):
    print( "Could not find the optimal solution" )
    exit(1)

# print optimal objective and optimal solution
print( "\nOptimal Objective: " + str( myModel.ObjVal ) )


NameError: name 'Model' is not defined

In [5]:
# some decision variables values
myVars = myModel.getVars()
result_df = pd.DataFrame(np.array([curVar.x for curVar in myVars if curVar.varName[0] == 'y']).\
             reshape(paper_data_matrix.T.shape))
result_df.index= paper_data.columns[1:]
result_df.columns =paper_data['paper']
result_df = result_df.T
result_df[result_df.columns[:10]]

,Referee1,Referee2,Referee3,Referee4,Referee5,Referee6,Referee7,Referee8,Referee9,Referee10
paper,,,,,,,,,,
Paper1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Paper2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Paper3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Paper4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
Paper5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
Paper67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Paper68,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Paper69,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#change excel cells color if it is not assigned 
wb = openpyxl.load_workbook('paper_data.xlsx')
sheet = wb.get_sheet_by_name('data')
myVars = myModel.getVars()
ori_paper_data = paper_data.values.copy()
for curVar in myVars:
    if curVar.varName[0] == 'y':
        i = int(curVar.varName[1:].split('_')[0])
        j = int(curVar.varName[1:].split('_')[1])
        if curVar.x == 0:
            #change the excel condition from "CONTAIN" to "BEGINNING WITH" before running this part
            sheet.cell(j+1, i+1).value = ' '+sheet.cell(j+1, i+1).value
            #sheet.cell(j+1, i+1).value = ''
            sheet.cell(j+1, i+1).fill = PatternFill(start_color='FFFFFF', fill_type = "solid")
wb.save('paper_data_updated.xlsx')

/Users/frank/anaconda3/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:296: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/frank/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  This is separate from the ipykernel package so we can avoid doing imports until
